# Climate Mode Calculation
- calculates climate mode index across CMIP6 models for intercomparison of yield response for phases

## Imports

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob as glob
import dask

## Processing SST (tos) data from the 18 underlying LOCA2 models
- need to regrid to common grid, everything is in .gn, not .gr

### Inputs

In [2]:
base_path = "/storage/group/pches/default/users/cta5244/CMIP6_tos/ssp370_omon_tos"
hist_path = "/storage/group/pches/default/users/cta5244/CMIP6_tos/hist_omon_tos"
model_paths = sorted(glob.glob(f"{base_path}/*"))

In [3]:
yearly_paths = sorted(glob.glob(f"{model_paths[0]}/*"))

### Computing

In [3]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="20GiB",
    walltime="02:00:00",
)

cluster.scale(jobs=5) 

Task exception was never retrieved
future: <Task finished name='Task-103540' coro=<Client._gather.<locals>.wait() done, defined at /storage/work/cta5244/.conda/envs/regrid_env/lib/python3.13/site-packages/distributed/client.py:2384> exception=AllExit()>
Traceback (most recent call last):
  File "/storage/work/cta5244/.conda/envs/regrid_env/lib/python3.13/site-packages/distributed/client.py", line 2393, in wait
    raise AllExit()
distributed.client.AllExit


In [4]:
from dask.distributed import Client

client = Client(cluster)

In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://146.186.150.12:8787/status,
Dashboard: http://146.186.150.12:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://146.186.150.12:44449,Workers: 0
Dashboard: http://146.186.150.12:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [42]:
def calculate_enso_index(model_path, save_out_path, lon_candidates=("lon","longitude","x","xh")):
    """
    given model_path, calculates enso index
    inputs 
    - model_path
    - save_out_path
    """
    yearly_paths = sorted(glob.glob(f"{model_path}/*"))
    # output string with gr not gn 
    new_list = ['gr' if item == 'gn' else item for item in yearly_paths[0].split("/")[-1].split("_")]
    output_unique_str = "_".join(new_list)
    # opening & regridding datasets
    ds_cmip6 = xr.open_mfdataset(yearly_paths)
    ds_cmip6.load()
    if 'lat' in ds_cmip6.coords:
        ds_cmip6 = ds_cmip6.rename({'lat':'latitude'})
        ds_cmip6 = ds_cmip6.rename({'lon':'longitude'})
        
    ds_cmip6 = ds_cmip6.assign_coords(longitude=(ds_cmip6['longitude'] % 360))

    regridder = xe.Regridder(ds_cmip6, era_ds, 'conservative', periodic=True)
    ds_regridded = regridder(ds_cmip6, keep_attrs = True)
    ds_regridded.to_netcdf(f"{save_out_path}/{output_unique_str}.nc")
    return


In [43]:
results = []
for model_path in model_paths:
    out = dask.delayed(save_regridded_model)(input_model = ds_first, model_path=model_path, save_out_path=save_out_path)
    results.append(out)

In [44]:
results = dask.compute(*results)

MemoryError: Unable to allocate 102. GiB for an array with shape (1032, 830305, 16) and data type float64

In [ ]:
for model_path in model_paths[2:10]:
    yearly_paths = sorted(glob.glob(f"{model_path}/*"))
    hist_paths = sorted(glob.glob(f"{hist_path}/{model_path.split('/')[-1]}/*"))
    # output string with gr not gn 
    new_list = ['gr' if item == 'gn' else item for item in yearly_paths[0].split("/")[-1].split("_")]
    output_unique_str = "_".join(new_list)
    # opening & regridding datasets
    ds_cmip6 = xr.open_mfdataset(yearly_paths + hist_paths)
    #ds_cmip6.load()
    if ('latitude' in ds_cmip6.coords) and (len(ds_cmip6.latitude.shape) == 1):
        ds_cmip6 = ds_cmip6.rename({'latitude':'lat'})
        ds_cmip6 = ds_cmip6.rename({'longitude':'lon'})
        ds_cmip6 = ds_cmip6.assign_coords(longitude=(ds_cmip6['lon'] % 360))
        lat = ds_cmip6['lat']
        lon = ds_cmip6['lon']
        lon2d, lat2d = xr.broadcast(lon, lat)
        mask_n4 = ((lat2d >= -5) & (lat2d <= 5) & (lon2d >= 160) & (lon2d <= 210))
    elif ('lat' in ds_cmip6.coords) and (len(ds_cmip6.lat.shape) == 1):
        ds_cmip6 = ds_cmip6.assign_coords(longitude=(ds_cmip6['lon'] % 360))
        lat = ds_cmip6['lat']
        lon = ds_cmip6['lon']
        lon2d, lat2d = xr.broadcast(lon, lat)
        mask_n4 = ((lat2d >= -5) & (lat2d <= 5) & (lon2d >= 160) & (lon2d <= 210))
    elif ('latitude' in ds_cmip6.coords) and (len(ds_cmip6.latitude.shape) == 2):
        ds_cmip6 = ds_cmip6.assign_coords(longitude=(ds_cmip6['latitude'] % 360))
        lon2d, lat2d = ds_cmip6.longitude, ds_cmip6.latitude
        mask_n4 = ((lat2d >= -5) & (lat2d <= 5) & (lon2d >= 160) & (lon2d <= 210))

    tos = ds_cmip6['tos'].where(mask_n4)
    weights = xr.ufuncs.cos(np.deg2rad(lat2d)).where(mask_n4)
    clim = tos.sel(time=slice('1991','2020')).groupby('time.month').mean('time')
    anom = tos.groupby('time.month') - clim

In [ ]:
yearly_paths + hist_paths

In [89]:
hist_paths

['/storage/group/pches/default/users/cta5244/CMIP6_tos/hist_omon_tos/EC-Earth3',
 '/storage/group/pches/default/users/cta5244/CMIP6_tos/hist_omon_tos/EC-Earth3-Veg']